In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import umap
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

sns.set(style='white', context='notebook', rc={'figure.figsize': (14, 10)})


In [4]:
df = pd.read_hdf("recoded_hapmap_ped_case_control_vcf.h5")
df_map = pd.read_hdf("df_together_vcf_map.h5")
print(len(df))


1656


In [3]:
df

id,rs11240777,rs6681049,rs4475691,rs72631889,rs13302982,rs2272757,rs13303010,rs6669800,rs28391282,rs28561399,...,rs909439,rs5983831,rs954657,rs7876575,rs3093463,rs9943032,rs9786255,rs41531144,index,group
0,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA20754,cases
1,1.0,2.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA20540,cases
2,1.0,2.0,1.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00150,cases
3,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00135,cases
4,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NA11843,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,0.0,2.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG01756,control
1652,0.0,2.0,0.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA10835,control
1653,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG01507,control
1654,1.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA07031,control


In [10]:
igsr_samples = pd.read_csv("IGSR/igsr_samples.tsv", delimiter="\t")
igsr_samples.head(n=2)

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00360,male,SAME124539,FIN,"Finnish,Finnish",EUR,"European Ancestry,West Eurasia (SGDP)","FIN,FinnishSGDP","1000 Genomes on GRCh38,Simons Genome Diversity..."
1,HG00132,female,SAME123061,GBR,British,EUR,European Ancestry,GBR,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


In [11]:
df.merge(igsr_samples, how="left", left_on="index", right_on="Sample name")


,rs11240777,rs6681049,rs4475691,rs72631889,rs13302982,rs2272757,rs13303010,rs6669800,rs28391282,rs28561399,...,group,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,cases,NA20754,male,SAME124173,TSI,Toscani,EUR,European Ancestry,TSI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,1.0,2.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,...,cases,NA20540,female,SAME125148,TSI,Toscani,EUR,European Ancestry,TSI,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,1.0,2.0,1.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,...,cases,HG00150,female,SAME124591,GBR,British,EUR,European Ancestry,GBR,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,...,cases,HG00135,female,SAME123068,GBR,British,EUR,European Ancestry,GBR,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."
4,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,cases,NA11843,male,SAME125140,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,0.0,2.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,1.0,...,control,HG01756,male,SAME124988,IBS,Iberian,EUR,European Ancestry,IBS,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1652,0.0,2.0,0.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,...,control,NA10835,male,SAME124547,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes 30x on GRCh38,1000 Genomes phase ..."
1653,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,control,HG01507,female,SAME124948,IBS,Iberian,EUR,European Ancestry,IBS,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1654,1.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,...,control,NA07031,female,SAME122860,CEU,CEPH,EUR,European Ancestry,CEU,"1000 Genomes 30x on GRCh38,1000 Genomes phase ..."
